#Combinatorial Optimization Problems on Variational Quantum Eigensolver(VQE) and QAOA

Combinatorial optimization problem is one of the usage of variational algorithm. Who want to start learning quantum-classical hybrid algorithm on quantum computing, it may be a good subject to strat.

Combinatorial optimization problem is a problem to find the best solution by solving a minimized optimization problem. If you want to solve a social problem, to formulate this into a combination of binary number 0 and 1 and give some constraint on it.

##Formulation of combinatorial optimization problem and VQE
The formulation think the hamiltonian as a cost function for social problems and try to find the best answer from the qubits. The main step is,

1. Use ising model which is a basic physics model
2. Use combination of Z as a hamiltonian
3. More practically we use QUBO for the formulation which is 0 and 1 instead of -1 and +1 of ising model

Usually we rarely use VQE without qaoa ansatz for the combinatorial optimization problem, but this time we just set a simple 1qubit problem.

```python
h = -Z(0) - Z(0)*Z(1)
```

The number after Z is the number of qubits. Here we are using 0th qubit and 1st qubit in the quantum circuit. and the coefficients in front of Z is important.

The coefficient of Z(0) is a bias.
The coefficient of Z(0)*Z(1) is a weight.

Now we have both value of -1 as each coefficient.

Z takes -1 or +1 as a expectation value. If the hamiltonian h takes smaller value it will be the answer. 

Here we just check the whole answer in the table.

Z(0) | Z(1) | h
--:|:----:|:--
-1|-1|0
-1|1|2
1|-1|0
1|1|-2

The hamiltonian takes the minimu value of -2 when Z(0)=1 and Z(1)=1. VQE solve this automatically. This time as an ansatz we used 4 parameters of a,b,c,d. a, b for the 0th qubit, and c, d for the 1st qubit.

Now we check it on blueqat, first we need to install blueqat and give this code.

In [0]:
!pip3 install blueqat

     |████████████████████████████████| 51kB 1.6MB/s 


In [0]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.vqe import AnsatzBase, Vqe

class OneQubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian.to_expr(), 4)
        self.step = 1

    def get_circuit(self, params):
        a, b, c, d = params
        return Circuit().ry(a)[0].rz(b)[0].ry(c)[1].rz(d)[1]


# hamiltonian is important
h = -Z(0) - Z(0)*Z(1)
runner = Vqe(OneQubitAnsatz(h))
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

Result by VQE
-1.99999508430852


Now we get -2 as the expectation value of minimum value.

For practical use we usually solve much bigger problems. Now we can see this problem as a kind of classification problem which bias of Z(0) shows that Alice belongs to the group1 and weight of Z(0)*Z(1) shows that Alice and Bob belong to the same group as a kind of max-cut problem.

Here we used Z for operator, now we use a technique to transform it into 01 binary number.


##Formulation with QUBO
If use Z in the hamiltoinan Z takes -1 or +1 as an expectaion value. But it is little bit uncomfortable for the social problems. Usually we use 01 binary number so, we just transform hamiltonian into binary number using,

$$
q = \frac{Z + 1}{2}
$$

This is the transform of -1 to 0 and we now can use QUBO as a formulation.

##QUBO programming
Let's use qubo on blueqat. Blueqat has a function for qubo. Now we can write the qubo form as,

```python
h = -3*q(0)-3*q(1)-2*q(0)*q(1)
```

This hamiltonian obviously takes the minimum value of -8 when q(0)=1 and q(1)=1. By solving on VQE we may get the similar result.

In [0]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe

class QubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian, 4)
        self.step = 1

    def get_circuit(self, params):
        a, b, c, d = params
        return Circuit().ry(a)[0].rz(b)[0].ry(c)[1].rz(d)[1]

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
h = h.to_expr().simplify()
runner = Vqe(QubitAnsatz(h))
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

Result by VQE
-7.986060449670348
Result by numpy
-8.0


We correctly the result. This time we just used 2qubits. But actually it is very difficult to get the right answer on bigger number of qubits. To compute much more efficiently we are using a new ansatz called QAOA ansatz for combinatorial optimization problem.

##QAOA
QAOA has a special form of ansatz especially to solve combinatorial optimization problem using the similar step of VQE. Now we try to use QAOA(Quantum Approximate Opitmization Alogirthm) in this chapter.

##2-5-1 Quantum Adiabatic Algorithm
QAA is an algorithm to change the state vector adiabatically to keep the ground state of the hamiltonian through the time evolution.

We set the initial hamiltonian as $H_{start}$ and the final Hamiltonian as $H_{fin}$. t is time and T is the whole step of schedule.

$$
H_{temp} = (1-\frac{t}{T})H_{start} + \frac{t}{T}H_{fin}
$$

The hamiltoinan adiabatically changed according to the time t and the eigenstate of state vector keep the ground state if T is enough big.

$$
H_{temp}\mid \psi \rangle = E_{0temp}\mid \psi \rangle
$$

Time evolution is,

$$
\mid \psi_{t+1} \rangle = U \mid \psi_t \rangle = e^{-iHt}  \mid \psi_t \rangle
$$


##QAOA
QAOA is basically using the idea of adiabatic process and transform it into a variational algorithm as ansatz of hamiltonian.

![https___qiita-user-contents.imgix.net_https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Fe10f1843-cc16-cdfe-e4a6-e2fbaab6df9f.png_ixlib=rb-1.2.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/812819f5-0037-5a30-2690-8ed39c0b1a06.png)
https://app.quantumcomputing.com/

The first H on the circuit shows the initial eigen state of hamiltonian X. CX-Rz-CX shows the weight of hamiltonian and Rz shows the bias of the hamiltonian. Rx is a time evolution of initial hamiltonian X.

Let's just see the inside of QaoaAnsatz of blueqat. Hamiltonian consists of Z operator and automatically make a variational transformation of time evolution ansatz.


```python
class QaoaAnsatz(AnsatzBase):
    def __init__(self, hamiltonian, step=1, init_circuit=None):
        super().__init__(hamiltonian, step * 2)
        self.hamiltonian = hamiltonian.to_expr().simplify()
        if not self.check_hamiltonian():
            raise ValueError("Hamiltonian terms are not commutable")

        self.step = step
        self.n_qubits = self.hamiltonian.max_n() + 1
        if init_circuit:
            self.init_circuit = init_circuit
            if init_circuit.n_qubits > self.n_qubits:
                self.n_qubits = init_circuit.n_qubits
        else:
            self.init_circuit = Circuit(self.n_qubits).h[:]
        self.init_circuit.make_cache()
        self.time_evolutions = [term.get_time_evolution() for term in self.hamiltonian]

    def check_hamiltonian(self):
        """Check hamiltonian is commutable. This condition is required for QaoaAnsatz"""
        return self.hamiltonian.is_all_terms_commutable()

    def get_circuit(self, params):
        c = self.init_circuit.copy()
        betas = params[:self.step]
        gammas = params[self.step:]
        for beta, gamma in zip(betas, gammas):
            beta *= np.pi
            gamma *= 2 * np.pi
            for evo in self.time_evolutions:
                evo(c, gamma)
            c.rx(beta)[:]
        return c
```

On actual use, the library automatically do the calculation and you don't to implement a complicated formulation or time evolution.

Blueqat do most of the process and what you just need to do is to formulate the hamiltonian as a combinatorial optimization with binary number of 0 and 1.

Now we see an example of simple problem on qubo.

```python
cost = -3*q(0)-3*q(1)-2*q(0)*q(1)
```

This take obviously -8 as a result. It is very easy, now we can think about the social problem as binary number. q(0) and q(1) has -3 of bias and q(0)*q(1) has -2 as a weight.


In [0]:
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
step = 2

result = vqe.Vqe(vqe.QaoaAnsatz(h, step)).run()
print(result.most_common(12))

(((1, 1), 0.9552579305295982), ((0, 0), 0.02975250342490814), ((0, 1), 0.0074947830227465335), ((1, 0), 0.007494783022746532))


We get combination of (1,1) as the probability of 96%.